<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Data Regridding with CDO Step 2: Check Files and Clean for Resubmit</h1>
By: Kayla Besong, PhD
    <br>
Last Edited: 05/10/24
<br>
<br>    
<br>
This is the post-CDO regridding process that that identifies which files were not regridded or failed due to various reasons and provides some means for helping to fix that. This is the most chaotic of notebooks the sFWRD database as it really is a back of the napkin, fix what needs to be fixed, and send it back through CDO process. The processes used here might not solve problems that arose personally during the gridding process but may help identify a solution path.
<br>    
<br>
In some of cleaning steps there are multple code blocks, one for the original database and another for the processed harmonics analysis.

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import glob

# Find out if any files had issues in the CDO regridding process

and then fix them with various brut-force, bandaid, just-get-it-done code :)

In [3]:
def find_not_regridded(input_dir, output_dir):

    ''' Find out what files from the input directory did not make it to the output, regridded directory
    
    Inputs:

    input_dir: (str) path where input files are stored 
    output_fir: (str) path where output files are stored

    Output:

    bad: (list of strs) list of file names that were in the input directory but not the output directory    
    
    '''
    
    regridded_list = os.listdir(output_dir)                                        # List all files in the regridded database directory for model m
    og_list = os.listdir(input_dir)                                                # List all files in the original database directory for model m
    
    bad = []                                                                       # Initialize a list to keep track of files that are not regridded
    
    for i in og_list:                                                              # Iterate over each file in the original list
        if i not in regridded_list:                                                # Check if the file is not in the regridded list
            if i[-2:] == 'nc':                                                     # Ensure the file is a NetCDF file
                bad.append(i)                                                      # Add the file to the bad list if it's not regridded
    
    if len(bad) > 0:                                                               # Check if there are any bad files
        
        return bad                                                                  # Return the list of bad files
    
    else:
        print(f'No bad files for {m}')                                             # Print a message if there are no bad files for model m


## UFS_S2S

In [11]:
parent_in_parent = '../../../database_files/UFS_S2S/MPM'
parent_out_parent = '../../../UFS_Downscaled/Statistical/13km/MPM'

In [12]:
bad_13km = []

for s in sorted(os.listdir(parent_in_parent)):
    bad_13km.append(find_not_regridded(f'{parent_in_parent}/{s}', f'{parent_out_parent}/{s}'))

In [13]:
bad_13km

[['ffwiAVG_UFS_S2S_FORECAST_Daily_20110401.nc',
  'ffwiMIN_UFS_S2S_FORECAST_Daily_20110401.nc',
  'ffwiMAX_UFS_S2S_FORECAST_Daily_20110401.nc',
  'ffwi_UFS_S2S_FORECAST_Abs_20110401.nc',
  'vpdAVG_UFS_S2S_FORECAST_Daily_20110401.nc',
  'vpdMIN_UFS_S2S_FORECAST_Daily_20110401.nc',
  'vpdMAX_UFS_S2S_FORECAST_Daily_20110401.nc',
  'vpd_UFS_S2S_FORECAST_Abs_20110401.nc',
  'hdwAVG_UFS_S2S_FORECAST_Daily_20110401.nc',
  'hdwMIN_UFS_S2S_FORECAST_Daily_20110401.nc',
  'hdwMAX_UFS_S2S_FORECAST_Daily_20110401.nc',
  'hdw_UFS_S2S_FORECAST_Abs_20110401.nc'],
 ['ffwiAVG_UFS_S2S_FORECAST_Daily_20110415.nc',
  'ffwiMIN_UFS_S2S_FORECAST_Daily_20110415.nc',
  'ffwiMAX_UFS_S2S_FORECAST_Daily_20110415.nc',
  'ffwi_UFS_S2S_FORECAST_Abs_20110415.nc',
  'vpdAVG_UFS_S2S_FORECAST_Daily_20110415.nc',
  'vpdMIN_UFS_S2S_FORECAST_Daily_20110415.nc',
  'vpdMAX_UFS_S2S_FORECAST_Daily_20110415.nc',
  'vpd_UFS_S2S_FORECAST_Abs_20110415.nc',
  'hdwAVG_UFS_S2S_FORECAST_Daily_20110415.nc',
  'hdwMIN_UFS_S2S_FORECAST_Da

# Cleaning steps for failed files 

## NARR

In [ ]:
narr_grid = xr.open_dataset('air.sfc.1992.nc')

In [ ]:
narr_grid

In [ ]:
vars = ['ffwi', 'hdwi', 'Planetary_boundary_layer_height_surface',
        'Precipitation_rate_surface', 'Relative_humidity_height_above_ground',
        'Soil_moisture_content_layer_between_two_depths_below_surface_layer',
        'Temperature_height_above_ground', 
        'Total_precipitation_surface_3_Hour_Accumulation',
        'u-component_of_wind_height_above_ground',
        'v-component_of_wind_height_above_ground',
        'vpd', 'wdir', 'wspeed']


In [ ]:
vars_dict = {}

for v in vars:

    vars_dict[v.split('_')[0]] = v

In [ ]:
narr_dir = f'../database_files/NARR_ungrid/Climos/'

dsarr = []
ct =0
for f in os.listdir(narr_dir):

    if f[-2:] == 'nc':

        ds = xr.open_dataset(f'{narr_dir}/{f}')

        try:      
            v = vars_dict[f.split('/')[-1].split('_')[0]]                        # Attempt to extract the variable name from the file name
        
        except KeyError:      
            v = vars_dict[f.split('/')[-1].split('_')[0][0:-3]]                  # Adjust the variable extraction for special cases

        coords_to_keep = ['x', 'y', 'time']
        all_coords = list(ds.coords)
        
        if len(coords_to_keep) != len(all_coords):                        # Check if there are extra coordinates
            delete = [i for i in all_coords if i not in coords_to_keep]   # Identify coordinates to delete
            ds = ds.drop(delete)                                          # Drop unwanted coordinates
            
        narr_grid = xr.open_dataset('air.sfc.1992.nc')
        narr_grid = narr_grid.drop('air')        
        narr_grid = narr_grid.drop('time')
        
        narr_grid['time'] = ds['time']
        
        str_var = f'{v}_climo'
        
        vals_da = xr.DataArray(ds[str_var].values, dims=['time', 'y', 'x'], coords={'lat': narr_grid['lat'], 'lon': narr_grid['lon'], 'y': narr_grid['y'], 'x': narr_grid['x'], 'time': narr_grid['time']})
        narr_grid[str_var] = vals_da 
        
        narr_grid.to_netcdf(f'../database_files/NARR/Climos/cleaned/{f}')

        print(f)

## NAM

In [ ]:
def special_nam_handler_cdo(file, v):

    ''' This function takes problematic NAM files from the database 
        and adds a missing time dimension or grid that got lost or selects one level.
    
    
    Inputs:
    
    file: (xarray dataset) a NAM file opened with xarray
    v: (str) variable name
    
    Outputs:
    
    nam_grid: (xarray dataset) the new, cleaned version of the input 
    
    '''
    nam_grid = xr.open_dataset('../database_files/NAM/t2m_NAM_HISTORICAL_Abs_2012.nc')              # Open a NAM dataset to use its grid information and drop the 't2m' variable, always use a leap year!! 
    
    if 'Daily' in file.split('/')[-1].split('_'):                                                   # Check if the file represents daily data
        nam_grid = nam_grid.resample(time = '24H').mean('time')                                     # Resample the grid data to daily frequency and compute the mean
    
    df = xr.open_dataset(file)                                                                      # Open the dataset file
    
    nam_grid = nam_grid.isel(time = np.arange(0, len(df.time)))
    
    if v == 'sm':                                                                                   # Check if the variable is soil moisture ('sm')
        df = df.isel(depthBelowLandLayer = 0)                                                       # Select the first layer of soil moisture data
    
    nam_grid['time'] = df['time']                                                                   # Update the 'time' coordinate in the grid dataset to match the data file
    
    # Create a DataArray with the data values, dimensions, and coordinates aligned with the NAM grid
    df_vals_da = xr.DataArray(df[v].values, dims=['time', 'y', 'x'], coords={'latitude': nam_grid['latitude'], 'longitude': nam_grid['longitude'], 'y': nam_grid['y'], 'x': nam_grid['x'], 'time': nam_grid['time']})

    nam_grid = nam_grid.drop('t2m')
    nam_grid[v] = df_vals_da                                                                        # Assign the newly created DataArray to the NAM grid dataset under the variable v
    
    return nam_grid                                                                                 # Return the updated NAM grid dataset

In [ ]:
## check which files need a level selection 

ct = 0
for i in nam_no:
    x = xr.open_dataset(f'../database_files/NAM/{i}')
    print(list(x.dims), ct)
    ct+=1

In [ ]:
vars = os.listdir('../NAM')
vars.append('vpd')
vars.append('wspeed')
vars.append('wdir')
vars.append('hdwi')
vars.append('ffwi')

In [ ]:
vars_dict = {}

for v in vars:

    vars_dict[v.split('_')[0]] = v

In [ ]:
input_dir = '../database_files/NAM/'                                          # Define the input directory for NAM files

for f in nam_no:                                                              # Iterate over the list of NAM files
    if f[-2:] == 'nc':                                                        # Check if the file is a NetCDF file
        
        file = f'{input_dir}/{f}'                                             # Construct the full file path

        try:
            v = vars_dict[file.split('/')[-1].split('_')[0]]                  # Attempt to extract the variable name from the file name
        
        except KeyError:
            v = vars_dict[file.split('/')[-1].split('_')[0][0:-3]]            # Adjust the variable extraction for special cases

        out_nam = special_nam_handler_cdo(file, v)                            # Call a special handler function for NAM data processing

        coords_to_keep = ['time', 'latitude', 'longitude', 'y', 'x']          # Define the coordinates to keep
        out_nam_coords = list(out_nam.coords)                                 # List all coordinates in the output dataset
        
        if len(coords_to_keep) != len(out_nam_coords):                        # Check if there are extra coordinates
            delete = [i for i in out_nam_coords if i not in coords_to_keep]   # Identify coordinates to delete
            out_nam = out_nam.drop(delete)                                    # Drop unwanted coordinates

        out_nam.to_netcdf(f'../database_files/NAM_added_grid/{f}')            # Save the processed dataset to a new NetCDF file
        print(f)                                                              

In [ ]:
g_file = xr.open_dataset(f'../database_files/NAM/Climos/cape_AVG_NAM_HISTORICAL_Daily_climos_full_period.nc')

In [ ]:
%%time

nam_dir = f'../database_files/NAM/Climos'

for f in os.listdir(nam_dir):

    ds = xr.open_dataset(f'{nam_dir}/{f}')

    ds['latitude'] = g_file['latitude']
    ds['longitude'] = g_file['longitude']

    ds.to_netcdf(f'../database_files/NAM_added_grid/{f}')
    print(f)
    

## CONSU404

In [ ]:
def special_c404_handler_cdo(file, v):

    ''' This function takes problematic CONUS404 files from the database 
        and adds a non-generic grid.
    
    
    Inputs:
    
    file: (xarray dataset) a CONUS404 file opened with xarray
    v: (str) variable name
    
    Outputs:
    
    C404_grid: (xarray dataset) the new, cleaned version of the input 
    
    '''
    C404_grid = xr.open_dataset('../database_files/CONUS404/TD2_CONUS404_ANALYSIS_Abs_2012.nc')              # Open a C404 dataset to use its grid information and drop the 't2m' variable, always use a leap year!!! 
    
    if 'Daily' in file.split('/')[-1].split('_'):                                                            # Check if the file represents daily data
        C404_grid = C404_grid.resample(Time = '24H').mean('Time')                                            # Resample the grid data to daily frequency and compute the mean
            
    df = xr.open_dataset(file)                                                                               # Open the dataset file
    
    C404_grid = C404_grid.isel(Time = np.arange(0, len(df.Time)))
            
    C404_grid['Time'] = df['Time']                                                                           # Update the 'time' coordinate in the grid dataset to match the data file
            
    # Create a DataArray with the data values, dimensions, and coordinates aligned with the C404 grid
    df_vals_da = xr.DataArray(
    data=df[v].values,
    dims=['Time', 'south_north', 'west_east'],  
    coords={
        'Time': C404_grid.coords['Time'],  
        'XLAT': (('south_north', 'west_east'), C404_grid.coords['XLAT'].data),  
        'XLONG': (('south_north', 'west_east'), C404_grid.coords['XLONG'].data) 
    }
)
    C404_grid = C404_grid.drop('TD2')   
    C404_grid[v] = df_vals_da                                                                                # Assign the newly created DataArray to the C404 grid dataset under the variable v
            
    return C404_grid                                                                                         # Return the updated C404 grid dataset

In [ ]:
vars = os.listdir('../CONUS404')
vars.append('vpd')
vars.append('wspeed')
vars.append('wdir')
vars.append('hdwi')
vars.append('ffwi')

In [ ]:
vars_dict = {}

for v in vars:

    vars_dict[v] = v

In [ ]:
input_dir = '../database_files/CONUS404/'                                     # Define the input directory for C404 files

for f in c44_no:                                                              # Iterate over the list of C404 files
    if f[-2:] == 'nc':                                                        # Check if the file is a NetCDF file
        
        file = f'{input_dir}/{f}'                                             # Construct the full file path

        try:
            v = vars_dict[file.split('/')[-1].split('_')[0]]                  # Attempt to extract the variable name from the file name
        
        except KeyError:
            v = vars_dict[file.split('/')[-1].split('_')[0][0:-3]]            # Adjust the variable extraction for special cases

        out_C404 = special_c404_handler_cdo(file, v)                          # Call a special handler function for C404 data processing

        out_C404.to_netcdf(f'../database_files/CONUS404_added_grid/{f}')      # Save the processed dataset to a new NetCDF file
        print(f)                                                              

In [ ]:
for f in c404_no_climo[1:]:
    
    ex_file = xr.open_dataset(f'{parent_in}/T2_MAX_CONUS404_ANALYSIS_Daily_climos_full_period.nc')
    ex_file = ex_file.drop('T2_climo')

    
    ds = xr.open_dataset(f'{parent_in}/{f}')
    
    ex_file['Time'] = ds['Time']
    # ex_file['XTIME'] = ds['XTIME']

    ex_file['ffwi_climo'] = ds['ffwi_climo']
    
    ex_file.to_netcdf(f'../database_files/CONUS404_2/{f}')
    

## HRRR

In [ ]:
def special_HRRR_handler_cdo(file, v):

    ''' This function takes problematic HRRR files from the database 
        and add a non-generic grid.
    
    
    Inputs:
    
    file: (xarray dataset) a HRRR file opened with xarray
    v: (str) variable name
    
    Outputs:
    
    HRRR_grid: (xarray dataset) the new, cleaned version of the input 
    
    '''
    HRRR_grid = xr.open_dataset('../database_files/HRRR/t2m_HRRR_HISTORICAL_Abs_2017.nc')                    # Open a HRRR dataset to use its grid information and drop the 't2m' variable
    
    if 'Daily' in file.split('/')[-1].split('_'):                                                            # Check if the file represents daily data
        HRRR_grid = HRRR_grid.resample(time = '24H').mean('time')                                            # Resample the grid data to daily frequency and compute the mean
            
    df = xr.open_dataset(file)                                                                               # Open the dataset file

    HRRR_grid = HRRR_grid.drop('t2m')
    HRRR_grid['time'] = df['time']                                                                           # Update the 'time' coordinate in the grid dataset to match the data file

    df_vals_da = xr.DataArray(
    data=df[v].values,
    dims=['time', 'y', 'x'],  
    coords={
        'time': HRRR_grid.coords['time'],  
        'latitude': (('y', 'x'), HRRR_grid.coords['latitude'].data),  
        'longitude': (('y', 'x'), HRRR_grid.coords['longitude'].data),
        'x': HRRR_grid.coords['x'],
        'y': HRRR_grid.coords['y']
    }
)
       
    HRRR_grid[v] = df_vals_da                                                                                # Assign the newly created DataArray to the HRRR grid dataset under the variable v
            
    return HRRR_grid                                                                                         # Return the updated HRRR grid dataset

In [ ]:
added = os.listdir('../database_files/HRRR_added_grid/')
added

In [ ]:
vars = ['blh',
 'cape',
 'mstav',
 'u10',
 'v10',
 'gust',
 'prate',
 'tp',
 't2m',
 'd2m',
 'wspeed',
 'wdir',
 'hdwi',
 'vpd',
 'ffwi', 'rh']


In [ ]:
vars_dict = {}

for v in vars:

    vars_dict[v] = v

In [ ]:
input_dir = '../database_files/HRRR/'                                         # Define the input directory for HRRR files
test_files = []
for f in hrrr_no:                                                             # Iterate over the list of HRRR files
    if f[-2:] == 'nc':                                                        # Check if the file is a NetCDF file
        
        file = f'{input_dir}/{f}'                                             # Construct the full file path

        try:
            v = vars_dict[file.split('/')[-1].split('_')[0]]                  # Attempt to extract the variable name from the file name
        
        except KeyError:
            v = vars_dict[file.split('/')[-1].split('_')[0][0:-3]]            # Adjust the variable extraction for special cases

        t = xr.open_dataset(file)
        v = list(t.variables)[0]

        out_HRRR = special_HRRR_handler_cdo(file, v)                          # Call a special handler function for HRRR data processing

        out_HRRR.to_netcdf(f'../database_files/HRRR_added_grid/{f}')          # Save the processed dataset to a new NetCDF file
        print(f)                                                              

In [ ]:
for f in hrrr_no_climo:
    
    ex_file = xr.open_dataset(f'{parent_in}/t2m_HRRR_HISTORICAL_Abs_climos_full_period.nc')
    ex_file = ex_file.drop('t2m_climo')

    
    ds = xr.open_dataset(f'{parent_in}/{f}')
    
    ex_file['time'] = ds['time']

    ex_file['ffwi_climo'] = ds['ffwi_climo']
    
    ex_file.to_netcdf(f'../database_files/HRRR2/{f}')
    